In [1]:
import os
import time
import random
import numpy as np
from math import sqrt
from ElipticCurves import ElipticCurve
from GF import generate_gf_class
from point import generate_point_class

In [2]:
%load_ext autoreload
%autoreload 2

## TEST BASELINE

In [3]:
n = 0x400000000000000000002BEC12BE2262D39BCF14D
d = 0x183F60FDF7951FF47D67193F8D073790C1C9B5A3E
hT = 0x09C9C44277910C9AAEE486883A2EB95B7180166DDF73532EEB76EDAEF52247FF
Ld = 512
Ln = n.bit_length()
m = 163
f = (1 << m) + (1 << 7) + (1 << 6) + (1 << 3) + 1
GF = generate_gf_class(m, f)
A, B = GF(1), GF(0x5FF6108462A2DC8210AB403925E638A19C1455D21)
Point = generate_point_class(GF, A, B)

In [4]:
curve = ElipticCurve(m, f, 1, 0x5FF6108462A2DC8210AB403925E638A19C1455D21, n, Ld)

In [5]:
P = Point(GF(0x72D867F93A93AC27DF9FF01AFFE74885C8C540420), GF(0x0224A9C3947852B97C5599D5F4AB81122ADC3FD9B))

### Signature generation

In [6]:
Q = -P.mul(d)

In [7]:
T, D = curve.sign(None, d)
assert curve.check_signature(T, D, Q), "bad signature!"

(6122365347637220882231788004894154459170679499764, 4483820909583776758787392988414727874640243892540) (10490415459366762209720328616843430641869299909664, 195769550360468726379426365966603649921409351067) 5261011074134316233885959999361596087865054000733
11136500872233882832996241198076904042117981496175
WHAHAHHAHHAHA


AssertionError: bad signature!

In [ ]:
0x274EA2C0CAA014A0D80A424F59ADE7A93068D08A7

In [9]:
assert Q == Point(GF(0x057DE7FDE023FF929CB6AC785CE4B79CF64ABDC2DA),GF(0x3E85444324BCF06AD85ABF6AD7B5F34770532B9AA))

In [8]:
h = GF(0x03A2EB95B7180166DDF73532EEB76EDAEF52247FF)

In [9]:
assert GF.to_int(GF(0x09C9C44277910C9AAEE486883A2EB95B7180166DDF73532EEB76EDAEF52247FF), Ln) == 0x03A2EB95B7180166DDF73532EEB76EDAEF52247FF

In [10]:
e = 0x1025E40BD97DB012B7A1D79DE8E12932D247F61C6

In [11]:
eP = P.mul(e)

In [12]:
assert eP==Point(GF(0x42A7D756D70E1C9BA62D2CB43707C35204EF3C67C),GF(0x5310AE5E560464A95DC80286F17EB762EC544B15B))

In [13]:
Fe = eP.x

In [14]:
y = h * Fe

In [15]:
assert y == GF(0x274EA2C0CAA014A0D80A424F59ADE7A93068D08A7)

In [16]:
r = GF.to_int(y, Ln)

In [17]:
s = e + d * r % n

In [18]:
assert s == 0x2100D86957331832B8E8C230F5BD6A332B3615ACA

In [19]:
D = ElipticCurve.concat_to_signature(r, s, Ld)

In [20]:
assert D == 0x000000000000000000000002100D86957331832B8E8C230F5BD6A332B3615ACA00000000000000000000000274EA2C0CAA014A0D80A424F59ADE7A93068D08A7

### Done

In [21]:
old_r, old_s = r, s
old_y = y

### Checking

In [22]:
r, s = ElipticCurve.split_signature(D, Ld)

In [23]:
assert (r, s) == (old_r, old_s)

In [24]:
R = P.mul(s) + Q.mul(r)

In [25]:
assert R == Point(GF(0x42A7D756D70E1C9BA62D2CB43707C35204EF3C67C),GF(0x5310AE5E560464A95DC80286F17EB762EC544B15B))

In [26]:
y = h * R.x

In [27]:
assert y == old_y

In [28]:
r_hat = GF.to_int(y, Ln)

In [29]:
assert r == r_hat

## Test my variant

In [3]:
m = 167
f = 2**167 + 2**6 + 1
A, B = 1, 0x6EE3CEEB230811759F20518A0930F1A4315A827DAC
n = 0x3FFFFFFFFFFFFFFFFFFFFFB12EBCC7D7F29FF7701F
Ln = n.bit_length()
Ld = 512
# assert Ld >= 2 * Ln

In [4]:
curve = ElipticCurve(m, f, A, B, n, Ld)

In [5]:
GF = generate_gf_class(m, f)
GF8 = generate_gf_class(8, (1 << 8) + (1 << 4) + (1 <<3) + (1 << 1) + 1)

### Test GF multiplication

In [6]:
a = GF8(0x53)
b = GF8(0xCA)
assert a * b == GF8(1)

### Test pow

In [7]:
for _ in range(10):
    n1 = GF.random() 
    exp = random.randint(1, 100)
    
    p = n1
    for i in range(exp - 1):
        p *= n1
    assert n1.pow(exp) == p

### Test GF inverse

In [8]:
for _ in range(10):
    number = GF.random() 
    assert number * number.inverse() == GF(1) or number == GF(0)

### Test point on curve

In [9]:
for _ in range(10):
    curve.point_on_curve(curve.random_point())

## Test signing

In [10]:
for _ in range(10):
    print("Good...")
    private_k = curve.gen_private_k()
    public_k = curve.gen_public_k(private_k)
    
    message = os.urandom(10000)
    T, D = curve.sign(message, private_k)
    assert curve.check_signature(T, D, public_k), "bad signature!"

Good...


AssertionError: bad signature!